In [53]:
import os
import win32com.client
from pywinauto import application
import time
import pandas as pd

# from datetime import time


def connect(reconnect=True):
    # 재연결이라면 기존 연결을 강제로 kill
    if reconnect:
        try:
            os.system('taskkill /IM ncStarter* /F /T')
            os.system('taskkill /IM CpStart* /F /T')
            os.system('taskkill /IM DibServer* /F /T')
            os.system('wmic process where "name like \'%ncStarter%\'" call terminate')
            os.system('wmic process where "name like \'%CpStart%\'" call terminate')
            os.system('wmic process where "name like \'%DibServer%\'" call terminate')
        except:
            pass

    CpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

    if CpCybos.IsConnect:
        print('already connected.')

    else:
        app = application.Application()
        app.start(
            'C:\Daishin\Starter\\ncStarter.exe /prj:cp /id:{id} /pwd:{pwd} /pwdcert:{pwdcert} /autostart'.format(
                id='gnxm37', pwd='muj212^^', pwdcert='gnxm5649..')
        )
        # 연결 될때까지 무한루프
        while True:
            if CpCybos.IsConnect:
                break
            time.sleep(1)

        print('connected.')
    return CpCybos


def _get_ohlcv_by_date(StockChart, code, last_date, start_date, chart_gb, modified='1', field=(0, # 날짜
                                                                                               2, # 시가
                                                                                               3, # 고가
                                                                                               4, # 저가
                                                                                               5, # 종가
                                                                                               8, # 거래량
                                                                                               9, # 거래대금
                                                                                               12, # 상장주식수
                                                                                               13, # 시가총액
                                                                                               16, # 외국인보유수량
                                                                                               17, # 외국인보유비율
                                                                                               20, # 기관순매수
                                                                                               21, # 기관누적순매수
                                                                                               23 # 등락률
                                                                                               )):
    StockChart.SetInputValue(0, code)
    StockChart.SetInputValue(1, ord('1'))
    StockChart.SetInputValue(2, last_date)
    StockChart.SetInputValue(3, start_date)
    StockChart.SetInputValue(5, field)
    StockChart.SetInputValue(6, ord(chart_gb))
    StockChart.SetInputValue(9, ord(modified))
    StockChart.BlockRequest()
    return _get_df_from_stock_chart(StockChart)

def initIndexInfo(self):
    # 지표 기본 정보, 지표 이름 = 지표 라인 리스트
    self.indexList = {}
    self.indexList['지표선택 없음'] =['없음']
    self.indexList['이동평균(라인1개)'] =['이동평균']
    self.indexList['Stochastic Slow'] = ['SLOW K', 'SLOW D']
    self.indexList['MACD'] = ['MACD', 'SIGNAL', 'OSCILLATOR']
    self.indexList['RSI'] = ['RSI', 'SIGNAL']
    self.indexList['Binary Wave MACD'] = ['BWMACD','SIGNAL', 'OSCILLATOR']
    self.indexList['TSF'] = ['TSF','SIGNAL']
    self.indexList['ZigZag'] = ['ZigZag1', 'ZigZag2']
    self.indexList['Bollinger Band'] = ['Bol상', 'Bol하', 'Bol중']


def _get_df_from_stock_chart(StockChart):
    time.sleep(0.25)
    df = []
    num_col = StockChart.GetHeaderValue(1)
    num_row = StockChart.GetHeaderValue(3)
    code = StockChart.GetHeaderValue(0) # 초
    timess = StockChart.GetHeaderValue(18) # 초
    exFlag = StockChart.GetHeaderValue(19) # 예상체결 플래그
    cprice = StockChart.GetHeaderValue(13) # 현재가
    diff = StockChart.GetHeaderValue(2) # 대비
    cVol = StockChart.GetHeaderValue(17) # 순간체결수량
    vol = StockChart.GetHeaderValue(9) # 거래량
    
    for r_idx in range(num_row):
        curr_data = []

        for c_idx in range(num_col):
            curr_data.append(StockChart.GetDataValue(c_idx, r_idx))
        df.append(curr_data)

    df = pd.DataFrame(df, columns=['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'Accumulated sales volume' , 'Accumulated number of contract'])
    return df

def get_ohlcv_by_date(code, last_date, start_date, chart_gb, modified='1', field=(0,1,2,3,4,5,8,10,11)):
   

    print('get ohlcv by date')
    print(f'code: {code}\nstart_date: {start_date}\nlast_date: {last_date}')
    StockChart = win32com.client.Dispatch("CpSysDib.StockChart")

    df = _get_ohlcv_by_date(StockChart, code, last_date, start_date,
                            chart_gb, modified=modified, field=field)

    last_date = df['date'].iloc[-1]
    last_min = df['time'].iloc[-1]

    df_list = [df]

    while True:
        df = _get_ohlcv_by_date(StockChart, code, int(last_date), start_date,
                            chart_gb, modified=modified, field=field)

        curr_date = df['date'].iloc[-1]
        curr_min = df['time'].iloc[-1]

        if curr_date == last_date and curr_min == last_min:
            break

        last_date = curr_date
        last_min = curr_min

        df_list.append(df)

    df = pd.concat(df_list).drop_duplicates(['date', 'time']).reset_index(drop=True)
    
    return df


# 이미 연결되어있다면 재연결 x
CpCybos = connect(False)

code = 'A372910'
# 분봉은 최대 2년 데이터 제공
start_date = '20210622'
last_date = '20230630'
chart_gb = 'm'
df = get_ohlcv_by_date(code, last_date, start_date, chart_gb)
print(df)

already connected.
get ohlcv by date
code: A372910
start_date: 20210622
last_date: 20230630
            date  time   open   high    low  close   volume  \
0       20230630  1530   5910   5910   5910   5910     6475   
1       20230630  1520   5900   5900   5880   5880      465   
2       20230630  1519   5900   5900   5900   5900        1   
3       20230630  1518   5890   5890   5890   5890      365   
4       20230630  1517   5890   5890   5890   5890      830   
...          ...   ...    ...    ...    ...    ...      ...   
133678  20210817   905  15950  16000  15400  15550   359058   
133679  20210817   904  15400  16200  15100  15950   575322   
133680  20210817   903  17000  17150  15150  15350   853543   
133681  20210817   902  16900  17600  16500  16950  1254081   
133682  20210817   901  16050  16950  16050  16850  2339049   

        Accumulated sales volume  Accumulated number of contract  
0                         121871                           84414  
1                

In [54]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
stock_df = pd.DataFrame({'date':df['date'],'time':df['time'],'open':df['open'],'high':df['high'], 'low':df['low'],'close':df['close'], 'volume':df['volume'], 'Accumulated sales volume':df['Accumulated sales volume'] , 'Accumulated number of contract':df['Accumulated number of contract']})
stock_df

stock_df.to_csv('A372910.csv', index=False)

In [55]:
# print(4200*187979)